<a href="https://colab.research.google.com/github/TechKnight10/Summarization-model-using-Hugging-face-xlsum-and-ai4bharat-IndicBART/blob/main/summarizationwithIndicBART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import pandas as pd
from transformers import MBartForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import AlbertTokenizer, AutoTokenizer
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART-XLSum", do_lower_case=False, use_fast=False, keep_accents=True)
model = MBartForConditionalGeneration.from_pretrained("ai4bharat/IndicBART-XLSum")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/485 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

In [ ]:
file_path1 = r"urls_and_articles_summary_bengali.csv"
file_path2 = r"urls_and_articles_summary_gujarati.csv"
file_path3 = r"urls_and_articles_summary_hindi.csv"
file_path4 = r"urls_and_articles_summary_marathi.csv"
file_path5 = r"urls_and_articles_summary_tamil.csv"
file_path6 = r"urls_and_articles_summary_telugu.csv"

# Load each file and store in a separate DataFrame
df1 = pd.read_csv(file_path1, encoding='utf-8')
df2 = pd.read_csv(file_path2, encoding='utf-8')
df3 = pd.read_csv(file_path3, encoding='utf-8')
df4 = pd.read_csv(file_path4, encoding='utf-8')
df5 = pd.read_csv(file_path5, encoding='utf-8')
df6 = pd.read_csv(file_path6, encoding='utf-8')

# Preview the data for each file to confirm the "Article" column is loaded correctly
print("File 1 Preview:")
print(df1.head())

print("File 2 Preview:")
print(df2.head())

print("File 3 Preview:")
print(df3.head())

print("File 4 Preview:")
print(df4.head())

print("File 5 Preview:")
print(df5.head())

print("File 6 Preview:")
print(df6.head())

File 1 Preview:
                                  URL Source  \
0  https://www.bbc.com/bengali/news-56713767   
1  https://www.bbc.com/bengali/news-56533637   
2  https://www.bbc.com/bengali/news-56631353   
3  https://www.bbc.com/bengali/news-47189627   
4  https://www.bbc.com/bengali/news-43949363   

                                             Article  \
0  বাংলাদেশের ইসলামিক ফাউন্ডেশনও গত ১৪ই মার্চ ইসল...   
1  বিক্ষোভের সময় পুলিশ কাঁদানে গ্যাস ও রাবার বুল...   
2  ডুবে যাওয়া লঞ্চটি মুন্সিগঞ্জ থেকে নারায়ণগঞ্জ...   
3  ২০১৮ সালের নির্বাচন প্রত্যাখ্যান করেছে বিএনপি।...   
4  কার্টুনে দাম্পত্য জীবন মহারাষ্ট্রের কোলাপুরের ...   

                                             Summary  
0  ইসলামী শিক্ষাবিদ এবং যুক্তরাজ্যের জাতীয় স্বাস...  
1  বাংলাদেশে ভারতীয় প্রধানমন্ত্রী নরেন্দ্র মোদীর...  
2  বাংলাদেশে রাজধানী ঢাকার নিকটবর্তী নারায়ণগঞ্জে...  
3  বাংলাদেশে গত ৩০শে ডিসেম্বর অনুষ্ঠিত সংসদ নির্ব...  
4  বিয়ে টিকিয়ে রাখার ক্ষেত্রে স্বামী ও স্ত্রীর ...  
File 2 Preview:
            

In [ ]:
def get_last_full_stop(text):
    """Helper function to find the last full stop or '।' in the summary."""
    # Find the last full stop or '।'
    last_full_stop = max(text.rfind('.'), text.rfind('।'))
    if last_full_stop != -1:
        return text[:last_full_stop + 1]
    return text.strip() + '।'

def preprocess_input(article, tokenizer, max_input_length, device):
    """Preprocess the article, tokenize it, and ensure the length fits the model's input constraints."""
    inp = tokenizer(article + "</s><2hi>", add_special_tokens=True, return_tensors="pt", padding=True).input_ids
    print(f"Input tensor shape: {inp.shape}, on device: {inp.device}")

    # Truncate the input if it exceeds the max_input_length
    input_length = inp.shape[1]
    if input_length > max_input_length:
        inp = inp[:, :max_input_length]

    # Move the input tensor to the desired device (e.g., CPU or GPU)
    inp = inp.to(device)

    print(f"Processed input tensor shape: {inp.shape}, on device: {inp.device}")
    return inp

def summarize_article(article):
    try:
        # Preprocess the article
        inp = preprocess_input(article, tokenizer, model.config.max_position_embeddings, device)

        # Set the model to evaluation mode and move it to the device
        model.eval()
        model.to(device)

        # Generate the summary
        model_output = model.generate(
            inp,
            use_cache=True,
            do_sample=True,
            num_beams=8,
            temperature=0.8,
            top_k=10,
            top_p=0.2,
            max_length=90,
            min_length=35,
            length_penalty=0.9,
            early_stopping=False,
            pad_token_id=pad_id,
            bos_token_id=bos_id,
            eos_token_id=eos_id,
            decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc(' ')
        )

        # Decode the generated output
        decoded_output = tokenizer.decode(model_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)

        # Ensure the summary ends with '।'
        if not decoded_output.endswith('।'):
            decoded_output += '।'

        return decoded_output

    except Exception as e:
        print(f"Error: {e}")
        return "Error in summarization"


In [ ]:
df3['Summary'] = df3['Article'].apply(summarize_article)

print(df3[['Article', 'Summary']].head())
df3.to_csv("summarized_articles_hindi.csv", index=False, encoding='utf-8-sig')

Input tensor shape: torch.Size([1, 308]), on device: cpu
Processed input tensor shape: torch.Size([1, 308]), on device: cuda:0
Input tensor shape: torch.Size([1, 756]), on device: cpu
Processed input tensor shape: torch.Size([1, 756]), on device: cuda:0
Input tensor shape: torch.Size([1, 345]), on device: cpu
Processed input tensor shape: torch.Size([1, 345]), on device: cuda:0
Input tensor shape: torch.Size([1, 798]), on device: cpu
Processed input tensor shape: torch.Size([1, 798]), on device: cuda:0
Input tensor shape: torch.Size([1, 432]), on device: cpu
Processed input tensor shape: torch.Size([1, 432]), on device: cuda:0
Input tensor shape: torch.Size([1, 333]), on device: cpu
Processed input tensor shape: torch.Size([1, 333]), on device: cuda:0
Input tensor shape: torch.Size([1, 364]), on device: cpu
Processed input tensor shape: torch.Size([1, 364]), on device: cuda:0
Input tensor shape: torch.Size([1, 758]), on device: cpu
Processed input tensor shape: torch.Size([1, 758]), on

In [ ]:
df1['Summary'] = df1['Article'].apply(summarize_article)
print(df1[['Article', 'Summary']].head())
df1.to_csv("summarized_articles_bengali.csv", index=False, encoding='utf-8-sig')

Input tensor shape: torch.Size([1, 869]), on device: cpu
Processed input tensor shape: torch.Size([1, 869]), on device: cuda:0
Input tensor shape: torch.Size([1, 424]), on device: cpu
Processed input tensor shape: torch.Size([1, 424]), on device: cuda:0
Input tensor shape: torch.Size([1, 249]), on device: cpu
Processed input tensor shape: torch.Size([1, 249]), on device: cuda:0
Input tensor shape: torch.Size([1, 749]), on device: cpu
Processed input tensor shape: torch.Size([1, 749]), on device: cuda:0
Input tensor shape: torch.Size([1, 375]), on device: cpu
Processed input tensor shape: torch.Size([1, 375]), on device: cuda:0
Input tensor shape: torch.Size([1, 510]), on device: cpu
Processed input tensor shape: torch.Size([1, 510]), on device: cuda:0
Input tensor shape: torch.Size([1, 314]), on device: cpu
Processed input tensor shape: torch.Size([1, 314]), on device: cuda:0
Input tensor shape: torch.Size([1, 237]), on device: cpu
Processed input tensor shape: torch.Size([1, 237]), on

In [ ]:
df2['Summary'] = df2['Article'].apply(summarize_article)

print(df2[['Article', 'Summary']].head())
df2.to_csv("summarized_articles_Gujarati.csv", index=False, encoding='utf-8-sig')

Input tensor shape: torch.Size([1, 1485]), on device: cpu
Processed input tensor shape: torch.Size([1, 1024]), on device: cuda:0
Input tensor shape: torch.Size([1, 1066]), on device: cpu
Processed input tensor shape: torch.Size([1, 1024]), on device: cuda:0
Input tensor shape: torch.Size([1, 554]), on device: cpu
Processed input tensor shape: torch.Size([1, 554]), on device: cuda:0
Input tensor shape: torch.Size([1, 1095]), on device: cpu
Processed input tensor shape: torch.Size([1, 1024]), on device: cuda:0
Input tensor shape: torch.Size([1, 1494]), on device: cpu
Processed input tensor shape: torch.Size([1, 1024]), on device: cuda:0
Input tensor shape: torch.Size([1, 1140]), on device: cpu
Processed input tensor shape: torch.Size([1, 1024]), on device: cuda:0
Input tensor shape: torch.Size([1, 446]), on device: cpu
Processed input tensor shape: torch.Size([1, 446]), on device: cuda:0
Input tensor shape: torch.Size([1, 544]), on device: cpu
Processed input tensor shape: torch.Size([1,

In [ ]:
df4['Summary'] = df4['Article'].apply(summarize_article)

print(df4[['Article', 'Summary']].head())
df4.to_csv("summarized_articles_marathi.csv", index=False, encoding='utf-8-sig')

Input tensor shape: torch.Size([1, 181]), on device: cpu
Processed input tensor shape: torch.Size([1, 181]), on device: cuda:0
Input tensor shape: torch.Size([1, 887]), on device: cpu
Processed input tensor shape: torch.Size([1, 887]), on device: cuda:0
Input tensor shape: torch.Size([1, 1247]), on device: cpu
Processed input tensor shape: torch.Size([1, 1024]), on device: cuda:0
Input tensor shape: torch.Size([1, 912]), on device: cpu
Processed input tensor shape: torch.Size([1, 912]), on device: cuda:0
Input tensor shape: torch.Size([1, 482]), on device: cpu
Processed input tensor shape: torch.Size([1, 482]), on device: cuda:0
Input tensor shape: torch.Size([1, 152]), on device: cpu
Processed input tensor shape: torch.Size([1, 152]), on device: cuda:0
Input tensor shape: torch.Size([1, 849]), on device: cpu
Processed input tensor shape: torch.Size([1, 849]), on device: cuda:0
Input tensor shape: torch.Size([1, 1129]), on device: cpu
Processed input tensor shape: torch.Size([1, 1024])

In [ ]:
df5['Summary'] = df5['Article'].apply(summarize_article)

print(df5[['Article', 'Summary']].head())
df5.to_csv("summarized_articles_tamil.csv", index=False, encoding='utf-8-sig')

Input tensor shape: torch.Size([1, 524]), on device: cpu
Processed input tensor shape: torch.Size([1, 524]), on device: cuda:0
Input tensor shape: torch.Size([1, 482]), on device: cpu
Processed input tensor shape: torch.Size([1, 482]), on device: cuda:0
Input tensor shape: torch.Size([1, 1009]), on device: cpu
Processed input tensor shape: torch.Size([1, 1009]), on device: cuda:0
Input tensor shape: torch.Size([1, 801]), on device: cpu
Processed input tensor shape: torch.Size([1, 801]), on device: cuda:0
Input tensor shape: torch.Size([1, 1245]), on device: cpu
Processed input tensor shape: torch.Size([1, 1024]), on device: cuda:0
Input tensor shape: torch.Size([1, 238]), on device: cpu
Processed input tensor shape: torch.Size([1, 238]), on device: cuda:0
Input tensor shape: torch.Size([1, 524]), on device: cpu
Processed input tensor shape: torch.Size([1, 524]), on device: cuda:0
Input tensor shape: torch.Size([1, 943]), on device: cpu
Processed input tensor shape: torch.Size([1, 943])

In [ ]:
df6['Summary'] = df6['Article'].apply(summarize_article)

print(df6[['Article', 'Summary']].head())
df6.to_csv("summarized_articles_telugu.csv", index=False, encoding='utf-8-sig')

Input tensor shape: torch.Size([1, 1315]), on device: cpu
Processed input tensor shape: torch.Size([1, 1024]), on device: cuda:0
Input tensor shape: torch.Size([1, 1324]), on device: cpu
Processed input tensor shape: torch.Size([1, 1024]), on device: cuda:0
Input tensor shape: torch.Size([1, 1284]), on device: cpu
Processed input tensor shape: torch.Size([1, 1024]), on device: cuda:0
Input tensor shape: torch.Size([1, 584]), on device: cpu
Processed input tensor shape: torch.Size([1, 584]), on device: cuda:0
Input tensor shape: torch.Size([1, 1001]), on device: cpu
Processed input tensor shape: torch.Size([1, 1001]), on device: cuda:0
Input tensor shape: torch.Size([1, 1320]), on device: cpu
Processed input tensor shape: torch.Size([1, 1024]), on device: cuda:0
Input tensor shape: torch.Size([1, 482]), on device: cpu
Processed input tensor shape: torch.Size([1, 482]), on device: cuda:0
Input tensor shape: torch.Size([1, 840]), on device: cpu
Processed input tensor shape: torch.Size([1,